## Quantizing Home Advantage
Finding how much of a boost it is to teams playing at their home court

In [9]:
# Import packages
import pandas as pd # type: ignore
import numpy as np # type: ignore
import plotly.graph_objects as go # type: ignore
pd.set_option('future.no_silent_downcasting', True)

In [4]:
# Read pickle
nba_df = pd.read_pickle('pkl/nba_df.pkl')

# Replace values of W/L with 1 and 0
nba_df['W/L'] = nba_df['W/L'].replace({'L': 0, 'W': 1})
nba_df.head(5)

,Season,Team,G,Date,Home,Opp,W/L,Tm_Pts,Opp_Pts,Tm_FG,...,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF
0,2000,ATL,1,1999-11-02,0,WAS,0,87,94,31,...,13,16,.813,12,42,23,5,5,15,30
1,2000,ATL,2,1999-11-04,1,MIL,0,109,119,41,...,33,36,.917,12,38,24,15,6,11,25
2,2000,ATL,3,1999-11-06,1,CHI,1,113,97,44,...,24,35,.686,17,39,14,6,6,14,26
3,2000,ATL,4,1999-11-08,0,DEN,0,100,115,39,...,19,24,.792,22,49,28,6,15,7,23
4,2000,ATL,5,1999-11-10,0,VAN,0,97,102,39,...,13,16,.813,15,49,27,9,10,18,24


## Home Wins Percentage

It is well known that teams hold an advantage at their home court. This is to look at the extent they impact.

In [5]:
# Overall Home win percentage (by season)
nba_home = nba_df.groupby(['Season', 'Team', 'Home']).agg({'W/L': 'sum'})
nba_home = pd.merge(nba_home, 
                    nba_df.groupby(['Season', 'Team', 'Home']).agg({'G': 'count'}), 
                    on = ['Season', 'Team', 'Home'], 
                    how = 'inner')
nba_home['Win%'] = nba_home['W/L']/nba_home['G']
nba_home.reset_index(inplace=True)
nba_home.to_csv('pkl/nba_home_seasons.csv', index = False)
nba_home.head(10)

,Season,Team,Home,W/L,G,Win%
0,2000,ATL,0,7,41,0.170732
1,2000,ATL,1,21,41,0.512195
2,2000,BOS,0,9,41,0.219512
3,2000,BOS,1,26,41,0.634146
4,2000,CHI,0,5,41,0.121951
5,2000,CHI,1,12,41,0.292683
6,2000,CLE,0,10,41,0.243902
7,2000,CLE,1,22,41,0.536585
8,2000,DAL,0,18,41,0.439024
9,2000,DAL,1,22,41,0.536585


In [11]:
# Overall Home win percentage (overall)
nba_home_overall = nba_home.groupby(['Team', 'Home']).agg({'W/L': 'sum', 'G': 'sum'})
nba_home_overall['Win%'] = nba_home_overall['W/L']/nba_home_overall['G']
nba_home_overall.reset_index(inplace=True)
nba_home_overall.head(10)

# Plotly
# nba_home_overall

,Team,Home,W/L,G,Win%
0,ATL,0,325,963,0.337487
1,ATL,1,545,964,0.565353
2,BOS,0,466,966,0.482402
3,BOS,1,612,965,0.634197
4,BRK,0,185,441,0.419501
5,BRK,1,229,441,0.519274
6,CHI,0,361,961,0.37565
7,CHI,1,523,964,0.542531
8,CHO,0,138,357,0.386555
9,CHO,1,180,354,0.508475
